In [1]:
import prepare_data
import matplotlib.pyplot as plt
import numpy as np

Open data. All ECG have a lengh of 186. All the samples are cropped, downsampled and padded with zeroes if necessary to the fixed dimension of 188. Last column has the classification of the ecg : data.iloc[:,-1]

In [2]:
data = prepare_data.load_images_ecg()
mitbih = data['mitbih_train']
ptbdb = data['ptbdb']

As we checked in data_analysis, we have unbalenced data.
We will try first to train a model with the dataset as it is, check if there's a problem with unbalanced data, and try different approaches!

In [4]:
from keras.utils import to_categorical

#Prepare samples to train of mitbih
mitbid_ecg = mitbih.iloc[:,:186]
mitbid_classification = mitbih.iloc[:,-1]
x_train = mitbid_ecg
y_train = to_categorical(mitbid_classification)
x_test = data['mitbih_test'].iloc[:,:186]
y_test = to_categorical(data['mitbih_test'].iloc[:,-1])

In [5]:
# Let's try to create a simple 2 dense layers model with a final classification with softmax (model_dense_layers_v1)
import models
model = models.model_dense_layers_v1()
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
his = model.fit(x_train, y_train, batch_size=128, validation_data=(x_test, y_test) , epochs=80, verbose=True)
print('evaluation: ' )
loss , acc = model.evaluate(x_test, y_test)



Epoch 1/80


685/685 [==============================] - 5s 4ms/step - loss: 0.2720 - accuracy: 0.9250 - val_loss: 0.1789 - val_accuracy: 0.9524
Epoch 2/80
685/685 [==============================] - 3s 4ms/step - loss: 0.1449 - accuracy: 0.9599 - val_loss: 0.1516 - val_accuracy: 0.9571
Epoch 3/80
685/685 [==============================] - 3s 4ms/step - loss: 0.1178 - accuracy: 0.9668 - val_loss: 0.1277 - val_accuracy: 0.9641
Epoch 4/80
685/685 [==============================] - 3s 4ms/step - loss: 0.1029 - accuracy: 0.9710 - val_loss: 0.1302 - val_accuracy: 0.9606
Epoch 5/80
685/685 [==============================] - 3s 4ms/step - loss: 0.0952 - accuracy: 0.9730 - val_loss: 0.1264 - val_accuracy: 0.9649
Epoch 6/80
685/685 [==============================] - 3s 4ms/step - loss: 0.0911 - accuracy: 0.9739 - val_loss: 0.1058 - val_accuracy: 0.9715
Epoch 7/80
685/685 [==============================] - 3s 4ms/step - loss: 0.0827 - accuracy: 0.9760 - val_loss: 0.0980 - val_accuracy: 0.9728
Ep

In [6]:
import evaluation_functions

# Get predictions 
predictions = model.predict(x_test)
predictions = np.argmax(predictions, axis=1)
ground_truth = np.argmax(y_test, axis=1)

# Understand number of classes the model learned more.
evaluation_functions.print_categorical_acc(ground_truth=ground_truth, predictions=predictions)

685/685 [==============================] - 2s 2ms/step
Classe:0 | Total samples:18118 | Correct predictions: 17986 | 0.99271442764102
Classe:1 | Total samples:556 | Correct predictions: 426 | 0.7661870503597122
Classe:2 | Total samples:1448 | Correct predictions: 1373 | 0.9482044198895028
Classe:3 | Total samples:162 | Correct predictions: 124 | 0.7654320987654321
Classe:4 | Total samples:1608 | Correct predictions: 1574 | 0.9788557213930348


AS we see, the Classe 1 and classe 3 that have lower representation, have a accuracy of 76%
Let's try to oversample the samples

In [28]:
from imblearn.over_sampling import ADASYN

ada = ADASYN(sampling_strategy='all')

x_train_resample, y_train_resample = ada.fit_resample(x_train, np.argmax(y_train, axis=1))


In [29]:
# Let's try again the same exacct model, with a more balanced data
y_train_resample = to_categorical(y_train_resample)
model = models.model_dense_layers_v1()
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
his = model.fit(x_train_resample, y_train_resample, batch_size=128, validation_data=(x_test, y_test) , epochs=80, verbose=True)
print('evaluation: ' )
loss , acc = model.evaluate(x_test, y_test)

# Get predictions 
predictions = model.predict(x_test)
predictions = np.argmax(predictions, axis=1)
ground_truth = np.argmax(y_test, axis=1)

# Understand number of classes the model learned more.
evaluation_functions.print_categorical_acc(ground_truth=ground_truth, predictions=predictions)

Epoch 1/80


ValueError: in user code:

    File "c:\Users\maria\AppData\Local\anaconda3\envs\env_datascience\lib\site-packages\keras\src\engine\training.py", line 1401, in train_function  *
        return step_function(self, iterator)
    File "c:\Users\maria\AppData\Local\anaconda3\envs\env_datascience\lib\site-packages\keras\src\engine\training.py", line 1384, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Users\maria\AppData\Local\anaconda3\envs\env_datascience\lib\site-packages\keras\src\engine\training.py", line 1373, in run_step  **
        outputs = model.train_step(data)
    File "c:\Users\maria\AppData\Local\anaconda3\envs\env_datascience\lib\site-packages\keras\src\engine\training.py", line 1151, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "c:\Users\maria\AppData\Local\anaconda3\envs\env_datascience\lib\site-packages\keras\src\engine\training.py", line 1209, in compute_loss
        return 